<a href="https://colab.research.google.com/github/Yngridy08/Faculdade-ADS/blob/main/Trabalho_de_BIG_DATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [**QUESTÃO 1**](https://)

In [1]:
!wget https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip -O imdb-reviews-pt-br.zip
!unzip imdb-reviews-pt-br.zip
!rm imdb-reviews-pt-br.zip

--2024-12-12 17:23:48--  https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49549692 (47M) [application/zip]
Saving to: ‘imdb-reviews-pt-br.zip’

imdb-reviews-pt-br. 100%[===================>]  47.25M   105MB/s    in 0.5s    

2024-12-12 17:23:50 (105 MB/s) - ‘imdb-reviews-pt-br.zip’ saved [49549692/49549692]

Archive:  imdb-reviews-pt-br.zip
  inflating: imdb-reviews-pt-br.csv  


In [2]:
!pip install pyspark

In [3]:
from pyspark.sql import SparkSession

appName = "PySpark Trabalho de Big Data"
master = "local"

spark = SparkSession.builder.appName(appName).master(master).getOrCreate()

In [4]:
imdb_df = spark.read.csv('imdb-reviews-pt-br.csv',
                         header=True,
                         quote="\"",
                         escape="\"",
                         encoding="UTF-8")

In [5]:
def map1(x):
  if x['sentiment'] == 'neg':
    return ('id', int(x['id'])) # Covertendo o ID para inteiro
  else:
    return ('id', 0) # Retornando 0 para outros sentimentos

In [6]:
def reduceByKey1(x, y):
  # Soma os IDs para cada 'sentiment'
  return x + y

In [7]:
# Mapear as ocorrências de IDs dos filmes classificados como negativos
mapped_data = imdb_df.rdd.map(map1)

# Reduzir para somar as ocorrências
summed_counts = mapped_data.reduceByKey(reduceByKey1)

# Obter o total de ocorrências
total_count = summed_counts.filter(lambda x: x[0] == 'id').collect()[0][1]

# Mostrar resultado
print("A soma de ocorrências dos campos 'id' dos filmes classificados como negativos:", total_count)
print("RU 4424216")

A soma de ocorrências dos campos 'id' dos filmes classificados como negativos: 459568555
RU 4424216


# **QUESTÃO 2**


In [8]:
from pyspark.sql import SparkSession

appName = "PySpark Trabalho de Big Data"
master = "local"

spark = SparkSession.builder.appName(appName).master(master).getOrCreate()

In [9]:
imdb_df = spark.read.csv('imdb-reviews-pt-br.csv',
                         header=True,
                         quote="\"",
                         escape="\"",
                         encoding="UTF-8")

In [10]:
def map2(x):
  # Mapear a 'sentiment' como chave e uma tupla com a soma das palavras de cada texto como valor
  sentiment = x['sentiment']
  text_pt = x['text_pt']
  text_en = x['text_en']
  # Dividir o texto em palavras e contar o número de palavras
  words_pt = len(text_pt.split())
  words_en = len(text_en.split())

  return (sentiment, (words_pt, words_en))

In [11]:
def reduceByKey2(x,y):
  # Somar o número de palavras de cada texto português e inglês por 'sentiment'
  return (x[0] + y[0], x[1] + y[1])

In [14]:
# Mapear a 'sentiment' como chave e uma tupla com a soma das palavras de cada texto como valor
mapped_data = imdb_df.rdd.map(map2)

# Reduzir para somar o número total de palavras por 'sentiment'
word_counts = mapped_data.reduceByKey(reduceByKey2)

# Coletar e mostrar resultado
result = word_counts.collect()
eng_pt_difference = 0

for sentiment, word_count in result:
  print(f"Avaliação: {sentiment}, Total de Palavras em Português: {word_count[0]}, Total de Palavras em Inglês: {word_count[1]}")
  if(sentiment == 'neg'):
    eng_pt_difference = word_count[0] - word_count[1]

print(f"Diferença entre o total de palavras em inglês e português para a avaliação 'neg': {eng_pt_difference}")
print("RU 4424216")

Avaliação: neg, Total de Palavras em Português: 5455273, Total de Palavras em Inglês: 5400324
Avaliação: pos, Total de Palavras em Português: 5462204, Total de Palavras em Inglês: 5414747
Diferença entre o total de palavras em inglês e português para a avaliação 'neg': 54949
RU 4424216
